In [3]:
from monoHbbtools import Load
from coffea import util

In [4]:
from monoHbbtools import Utilities

In [5]:
Utilities.get_timestamp()

'2023-10-18 17:10:52'

In [7]:
file = util.load("Zjetsnunu.coffea")
file

{'Z1Jets_Nu_Nu': {'Cutflow': {'Total_Events': 3695915,
   'ReducedEvents': 489101,
   'ak4bJetsTight': 16890,
   'bbDiJets': 214},
  'Histograms': {'DiJet': Hist(Regular(100, 0, 500, label='Axis 0'), storage=Double()) # Sum: 212.0 (214.0 with flow)}},
 'MET': {'Cutflow': {'Total_Events': 11502275,
   'ReducedEvents': 662607,
   'ak4bJetsTight': 27196,
   'bbDiJets': 4361},
  'Histograms': {'DiJet': Hist(Regular(100, 0, 500, label='Axis 0'), storage=Double()) # Sum: 3705.0 (4361.0 with flow)}}}

In [18]:
Hist_List = [file["MET"]["Histograms"]["DiJet"], file["Z1Jets_Nu_Nu"]["Histograms"]["DiJet"] ]
Hist_List

[Hist(Regular(100, 0, 500, label='Axis 0'), storage=Double()) # Sum: 3705.0 (4361.0 with flow),
 Hist(Regular(100, 0, 500, label='Axis 0'), storage=Double()) # Sum: 212.0 (214.0 with flow)]

In [19]:
sorted(Hist_List, key = lambda x :  x.sum())

[Hist(Regular(100, 0, 500, label='Axis 0'), storage=Double()) # Sum: 212.0 (214.0 with flow),
 Hist(Regular(100, 0, 500, label='Axis 0'), storage=Double()) # Sum: 3705.0 (4361.0 with flow)]